# Prepare CNN Data

## Setup

In [12]:
# USE ACCOUNT: robmarty3@gmail.com
import ee
ee.Authenticate()
ee.Initialize()

Enter verification code:  4/1AX4XfWg0ml7vEHo3ZPn2KTW-oSPMTrY3_hQIj2c_LoazQYo7CE0kOY_H6xY



Successfully saved authorization token.


In [13]:
import numpy as np
import os, datetime
import pandas as pd
import itertools
import tensorflow as tf
import config as cf
import ee_utils as utils
import eeconvert
import time

#import geetools
#from geetools import ui, cloud_mask
#cloud_mask_landsatSR = cloud_mask.landsatSR()
#cloud_mask_sentinel2 = cloud_mask.sentinel2()

## Parameters

In [16]:
#224
SURVEY_NAME = 'DHS'
SATELLITE = 's2'
SKIP_IF_SCRAPED = True
CHUNK_SIZE = 1 # Number of observtaions to scrape in GEE at any given time
DROPBOX_DIR = cf.DROPBOX_DIRECTORY
GOOGLEDRIVE_DIR = cf.GOOGLEDRIVE_DIRECTORY

if SATELLITE == 's2':
    KERNEL_SIZE = 224
elif SATELLITE == 'l8':
    KERNEL_SIZE = 167

print(KERNEL_SIZE)

224


In [17]:
# Directory to store tfrecords
out_path = os.path.join(GOOGLEDRIVE_DIR, 
            'Data', 
            SURVEY_NAME, 
            'FinalData',
            'Individual Datasets',
            'cnn_' + SATELLITE,
            'tfrecords')

## Load Data

In [18]:
# Relies on VIIRs Values
survey_df = pd.read_csv(os.path.join(DROPBOX_DIR, 'Data', SURVEY_NAME, 'FinalData', 'Individual Datasets', 'data_for_cnn.csv'))
print(survey_df.shape)
print(survey_df.ntl_group.value_counts())

(63374, 8)
1    21907
2    13247
3    12022
4     9063
0     7135
Name: ntl_group, dtype: int64


In [19]:
# Remove observations that had issues
#survey_df = survey_df[survey_df['GID_2'] != "IND.18.1_1"]
survey_df = survey_df[survey_df['uid'] != 'CO201000003682']
survey_df = survey_df[survey_df['uid'] != 'CO201000003683']
survey_df = survey_df[survey_df['uid'] != 'CO201000004885']
survey_df = survey_df[survey_df['uid'] != 'IA201400180058']
survey_df = survey_df[survey_df['uid'] != 'IA201400180040']
survey_df = survey_df[survey_df['uid'] != 'IA201400300032']
survey_df = survey_df[survey_df['uid'] != 'IA201400010147']
survey_df = survey_df[survey_df['uid'] != 'NM201300000001']
survey_df = survey_df[survey_df['uid'] != 'CO201000002924']
survey_df = survey_df[survey_df['uid'] != 'CO201000002524']
survey_df = survey_df[survey_df['uid'] != 'CO201000001170']
survey_df = survey_df[survey_df['uid'] != 'CO201000003554']
survey_df = survey_df[survey_df['uid'] != 'GA201200000310']
survey_df = survey_df[survey_df['uid'] != 'IA201400180104'] # maybe OK?
survey_df = survey_df[survey_df['uid'] != 'IA201400180012']
survey_df = survey_df[survey_df['uid'] != 'IA201400180011']
survey_df = survey_df[survey_df['uid'] != 'IA201400180048']
survey_df = survey_df[survey_df['uid'] != 'IA201400300002']
survey_df = survey_df[survey_df['uid'] != 'IA201400130192']
survey_df = survey_df[survey_df['uid'] != 'IA201400140845']
survey_df = survey_df[survey_df['uid'] != 'IA201400010076']

# List of TF Records
tf_record_list = list(np.unique(survey_df.tfrecord_name))

In [20]:
# If skip already scraped, remove existing tfrecords from tf_record_list
if SKIP_IF_SCRAPED:
    tf_records_exist = os.listdir(out_path)
    tf_record_list = [x for x in tf_record_list if x not in tf_records_exist]
    
print(len(tf_record_list))

793


In [ ]:
# DONT SKIP ERRORS
if True:
    ### Loop through all tfrecords
    for tfr_i in tf_record_list:

        # Sometimes we get computational time out errors. If occurs, just skip and go to next.
        # We can then go back and rescrape missed ones.

        survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
        year_i = survey_df_yeari['year'].iloc[0]

        ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
        survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

        print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

        proto_examples_all = []
        for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
            #time.sleep(6)
            
            ## Sometimes we hit a memory error; try until we don't hit that
            
            # TODO: could say: try 3 times?
            try_extract_data = True
            while try_extract_data:
                try:

                    print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

                    survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

                    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
                    proto_examples_all.extend(proto_examples_i)
                    
                    try_extract_data = False
                    
                except:
                    print("Error!")
                    time.sleep(10)
                    pass
            
        ### Save data as tf record
        out_path_i = os.path.join(out_path, tfr_i)
        print(out_path_i)
        with tf.io.TFRecordWriter(out_path_i) as writer:
            for tf_example in proto_examples_all:
                writer.write(tf_example.SerializeToString())

        print("Success \o/")

/Users/robmarty/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  del sys.path[0]


Putting 57 observations into forcnn_AL_3_1_all.tfrecord
Observation: 0/57
Observation: 1/57
Observation: 2/57
Observation: 3/57
Observation: 4/57
Observation: 5/57
Observation: 6/57
Observation: 7/57
Observation: 8/57
Observation: 9/57
Observation: 10/57
Observation: 11/57
Observation: 12/57
Observation: 13/57
Observation: 14/57
Observation: 15/57
Observation: 16/57
Observation: 17/57
Observation: 18/57
Observation: 19/57
Observation: 20/57
Observation: 21/57
Observation: 22/57
Observation: 23/57
Observation: 24/57
Observation: 25/57
Observation: 26/57
Observation: 27/57
Observation: 28/57
Observation: 29/57
Observation: 30/57
Observation: 31/57
Observation: 32/57
Observation: 33/57
Observation: 34/57
Observation: 35/57
Observation: 36/57
Observation: 37/57
Observation: 38/57
Observation: 39/57
Observation: 40/57
Observation: 41/57
Observation: 42/57
Observation: 43/57
Observation: 44/57
Observation: 45/57
Observation: 46/57
Observation: 47/57
Observation: 48/57
Observation: 49/57
Obse

In [ ]:
if False:
    ### Loop through all tfrecords
    for tfr_i in tf_record_list:

        # Sometimes we get computational time out errors. If occurs, just skip and go to next.
        # We can then go back and rescrape missed ones.
        try:

            survey_df_yeari = survey_df[survey_df['tfrecord_name'] == tfr_i]
            year_i = survey_df_yeari['year'].iloc[0]

            ### Loop through chunks within tfrecord (can only pull so much data from GEE at a time)
            survey_df_yeari['chunk_id'] = utils.chunk_ids(survey_df_yeari.shape[0], CHUNK_SIZE)

            print("Putting " + str(survey_df_yeari.shape[0]) + " observations into " + tfr_i)

            proto_examples_all = []
            for chunk_i in list(np.unique(survey_df_yeari.chunk_id)):
                #time.sleep(7)
                print("Observation: " + str(len(proto_examples_all)) + "/" + str(survey_df_yeari.shape[0]))

                survey_df_yeari_chunki = survey_df_yeari[survey_df_yeari['chunk_id'] == chunk_i]

                proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki, SATELLITE, KERNEL_SIZE, year_i)
                proto_examples_all.extend(proto_examples_i)

            ### Save data as tf record
            out_path_i = os.path.join(out_path, tfr_i)
            print(out_path_i)
            with tf.io.TFRecordWriter(out_path_i) as writer:
                for tf_example in proto_examples_all:
                    writer.write(tf_example.SerializeToString())

            print("Success \o/")

        except:
            time.sleep(15)
            pass

In [10]:
for row_i in range(0,10):
    print(row_i)
    proto_examples_i = utils.prep_cnn_np(survey_df_yeari_chunki.iloc[[row_i]], SATELLITE, KERNEL_SIZE, year_i)

0
1
2
3
4


IndexError: positional indexers are out-of-bounds

In [ ]:
survey_df_yeari_chunki.iloc[[4]]